Regression and Stochastic Gradient Descent
===================================



Regression consists in finding a model $f$ that depends on parameters $\theta$ and that verifies $f(\theta, x_i) = y_i$ for a given set of $(x_i, y_i)_{i=1..n}$. 

Most often, this can only be verified approximately, typically because there is noise or because the problem is over-determined.

In that case, regression can be recast as an optimization problem, for example by minimizing the loss: 

\begin{equation}
L = \sum_i \| y_i - f(\theta, x_i) \|_2^2
\end{equation}


In [ ]:
from __future__ import print_function
import numpy as np
from matplotlib import pyplot as plt
import torch
import numpy as np

from torch import nn
from torch import optim
import torch.nn.functional as F


# Regressing between 2 squares

Let's make 2 sets of points, both on a square, such that there is an affine transformation between them

In [ ]:
def gen_2squares(n): 
    a = np.random.rand(n) * 2 * np.pi
    x = np.vstack((np.cos(a), np.sin(a)))
    x = x / np.linalg.norm(x, axis=0, ord=1)
    y = np.vstack((np.cos(a + np.pi / 4), np.sin(a + np.pi / 4)))
    y = y / np.linalg.norm(y, axis=0, ord=np.inf)
    y[0] += 3
    x += np.random.randn(*x.shape) * 0.02
    y += np.random.randn(*x.shape) * 0.02
    return x.T, y.T
    

In [ ]:
x, y = gen_2squares(200)

In [ ]:
pyplot.plot(x[:, 0], x[:, 1], '+', label='x')
pyplot.plot(y[:, 0], y[:, 1], '.', label='y')
pyplot.axis('equal')
pyplot.legend()
pyplot.grid()

The affine transformation: the parameters are a 2x2 matrix and a 2D translation. We stack them together in a 2x3 matrix


In [ ]:
def f(theta, x):
    return torch.matmul(x, theta[:2, :]) + theta[2, :]

In [ ]:
xt = torch.from_numpy(x.astype('float32'))
yt = torch.from_numpy(y.astype('float32'))

# starting point
theta = torch.tensor([
    [1., 0.], 
    [0., 1.], 
    [0., 0.]
])

# set the learning rate
learning_rate = 0.1
for it in range(20):    
    
    # we will need a gradient wrt. x
    theta.requires_grad = True
    
    # call the function, record dependencies for the gradient
    y_cur = f(theta, xt)
    err = ((y_cur - yt) ** 2).sum(1).mean()
    
    print(err.item())
    
    # compute gradients
    err.backward()    
    
    # update current solution
    theta = theta.data - learning_rate * theta.grad


**Exercices:** 

- what is the error we are optimizing?

- visualize the intermediate solutions

- vary the learning rate. What is the range of paramters where this converges?

- vary the noise on the input data.

# Stochastic gradient descent

The optimization sees points in "mini-batches", ie. the sum $\sum_i \| y_i - f(\theta, x_i) \|_2^2$ is not computed as a whole but for a subset of points. The points are visited in a random order. 

In [ ]:
xt = torch.from_numpy(x.astype('float32'))
yt = torch.from_numpy(y.astype('float32'))

# starting point
theta = torch.tensor([
    [1., 0.], 
    [0., 1.], 
    [0., 0.]
])

# set the learning rate
learning_rate = 0.5
n = xt.shape[0]

for it in range(20):        
    # random order 
    perm = torch.randperm(n)
    errs = []
    for i0 in range(0, n, 10):
        
        # i0 is called the batch size
        theta.requires_grad = True
        
        # we handle this subset of points (a minibatch)
        xbatch = xt[perm[i0 : i0 + 10]]
        ybatch = yt[perm[i0 : i0 + 10]]
    
        # compute the partial loss on this mini-batch
        y_cur_batch = f(theta, xbatch)
        err = ((y_cur_batch - ybatch) ** 2).sum(1).mean()
        errs.append(err.item())
    
        # compute gradients
        err.backward()    

        # update current solution
        theta = theta.data - learning_rate * theta.grad

    print(np.mean(errs))

**Exercices**

- What do we observe in terms of learning speed?

- try to vary the batch size. Which one works best?

# Write it with a Module and an Optimizer object

An opimizer manages several parameters and does update them. 


A module is an object that can be called as a function that automatically identifies its 
parameters -- tensors with gradients


In [ ]:
class AffineTransform(nn.Module):
    
    def __init__(self):
        nn.Module.__init__(self)
        self.A = nn.Parameter(
            torch.tensor([
                [1., 0.], 
                [0., 1.]
            ])
        )
        self.b = nn.Parameter(torch.tensor([0., 0.]))

    def forward(self, x):
        # called when the object is used as a function
        return torch.matmul(x, self.A) + self.b


In [ ]:
xt = torch.from_numpy(x.astype('float32'))
yt = torch.from_numpy(y.astype('float32'))

# the "model" = function + parameters
af = AffineTransform()

# the optimizer will take care of the parameters
optimizer = optim.SGD(af.parameters(), lr=0.5)

n = xt.shape[0]

for it in range(20):        
    perm = torch.randperm(n)
    errs = []
    for i0 in range(0, n, 10):

        # manipulate through the optimizer object
        optimizer.zero_grad()

        xbatch = xt[perm[i0 : i0 + 10]]
        ybatch = yt[perm[i0 : i0 + 10]]
    
        # call the model with the batch
        y_cur_batch = af(xbatch)
        err = ((y_cur_batch - ybatch) ** 2).sum(1).mean()
        errs.append(err.item())
    
        # compute gradients
        err.backward()    
        
        # update current solution
        optimizer.step()

    print(np.mean(errs))

# Regressing between a square and... a circle!

This time we center the data on (0, 0) and remove the noise.  

In [ ]:
def gen_square_circle(n): 
    n4 = int(n/4)
    a = np.random.rand(n) * 2 * np.pi
    x = np.vstack((np.cos(a), np.sin(a)))
    x = x / np.linalg.norm(x, axis=0, ord=2)
    y = np.vstack((np.cos(a + np.pi / 4), np.sin(a + np.pi / 4)))
    y = y / np.linalg.norm(y, axis=0, ord=np.inf)
    return x.T, y.T
    

In [ ]:
x, y = gen_square_circle(200)

pyplot.plot(x[:, 0], x[:, 1], '+', label='x')
pyplot.plot(y[:, 0], y[:, 1], '.', label='y')
pyplot.axis('equal')
pyplot.legend()
pyplot.grid()

Obviously we need a non-linear model. One of the great principles of neural nets is that we have the combination of:

- large linear transformations: fast to evaluate and lots of parameters

- scalar non-linearities.

The combination has a great expressive power: many mappings can be expressed with it. Let's make a 2-layer neural net to find a reasonable mapping.

In [ ]:
class NonLinearTransform(nn.Module):
    
    def __init__(self):
        nn.Module.__init__(self)
        # object that encapsulates an affine transform, initialized randomly
        # the parameters of the affine transform are extracted automatically
        self.fc1 = nn.Linear(2, 8)
        self.fc2 = nn.Linear(8, 2)
        
    def forward(self, x):
        z = self.fc1(x)
        y = self.fc2(F.relu(z))
        return y

In [ ]:
xt = torch.from_numpy(x.astype('float32'))
yt = torch.from_numpy(y.astype('float32'))

# the "model" = function + parameters
nlt = NonLinearTransform()

# the optimizer will take care of the parameters
lr = 0.1
optimizer = optim.SGD(nlt.parameters(), lr=lr, momentum=0.9)

n = xt.shape[0]
all_errs = []
for it in range(200):        
    perm = torch.randperm(n)
    errs = []
    nlt.train()
    bs = 20
    for i0 in range(0, n, bs):

        # manipulate through the optimizer object
        optimizer.zero_grad()

        xbatch = xt[perm[i0 : i0 + bs]]
        ybatch = yt[perm[i0 : i0 + bs]]
    
        # call the model with the batch
        y_cur_batch = nlt(xbatch)
        err = ((y_cur_batch - ybatch) ** 2).sum(1).mean()
        errs.append(err.item())
    
        # compute gradients
        err.backward()    
        
        # update current solution
        optimizer.step()
    # lr /= 1.05
    optimizer = optim.SGD(nlt.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
    nlt.eval()
    with torch.no_grad():
        points.append(nlt(xt).numpy())
    all_errs.append(np.mean(errs))
    if it % 10 == 0: 
        print(it, all_errs[-1])

In [ ]:
plt.semilogy(all_errs)

**Exercices**:

- visualize the mapping on concentric circles

- generate more points, add noise 

- how can we improve the objective and mapping: use different learning rate? bigger hidden state? deeper network? different non-linearity?
